In [34]:
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
import datetime

In [4]:
from function_set import *

In [5]:
anue_url = 'https://api.cnyes.com/media/api/v1/newslist/category/headline'
eco_url = 'https://money.udn.com/rank/newest/1001/0/1?from=edn_navibar'
ctee_url = 'https://ctee.com.tw/livenews/aj'


In [36]:
class anue():
    def __init__(self, url, page:str, start:str = str(int((datetime.datetime.utcnow()-datetime.timedelta(1)).timestamp())), end:str =str(int(datetime.datetime.utcnow().timestamp())-1)):
        self.url = url
        self.page = page
        self.start = start
        self.end = end
        self.prefix = 'https://news.cnyes.com/news/id/'
        
    def get_response(self):
        payload = {
        'limit':'30',
        'startAt': self.start,
        'endAt': self.end,
        'page': self.page,
        'startAt': self.start,
        'endAt': self.end,
        'limit': '30'
        }
        self.payload = payload
        self.response = rq.get(self.url, params = self.payload)
    
    def get_total_pages(self):
        self.get_response()
        self.data = self.response.json()
        total = self.data['items']['total']
        per_page = self.data['items']['per_page']
        self.total_pages = int(total/per_page)
    
    def get_titles_urls_per_page(self):
        self.get_response()
        news_urls = []
        news_titles = []
        for i in self.response.json()['items']['data']:
            news_urls += [i['newsId']]
            news_titles += [i['title']]
        data = {
            'news_urls':news_urls,
            'news_titles':news_titles
            }
        return pd.DataFrame(data)
    
    def get_all_titles_urls(self):
        self.get_total_pages()
        data = pd.DataFrame()
        for i in range(1, self.total_pages+1):
            self.page = i
            temp = self.get_titles_urls_per_page()
            data = pd.concat([data, temp])
        data = data.reset_index(drop=True)
        data['news_urls'] = data['news_urls'].apply(lambda x:self.prefix+str(x))
        return data



In [37]:
tirgger = anue(anue_url, 1)

In [38]:
data_anue = tirgger.get_all_titles_urls()

In [39]:
data_anue.drop_duplicates()

,news_urls,news_titles
0,https://news.cnyes.com/news/id/5095700,〈美股早盤〉科技股撐腰 主要指數開高 輝達漲逾12%
1,https://news.cnyes.com/news/id/5095969,建商富旺去年大虧9.78億元 每股虧6.35元 擬減資38%彌補虧損
2,https://news.cnyes.com/news/id/5095982,〈財報〉Nikola上季僅交車20輛電動卡車 營收遠低於預期
3,https://news.cnyes.com/news/id/5095701,美國上周初領失業金人數下滑 去年第四季GDP向下修正
4,https://news.cnyes.com/news/id/5095967,渣打：今年經濟成長預估值維持2.2%不變 央行3月利率按兵不動機率高
...,...,...
265,https://news.cnyes.com/news/id/5093065,Fed副主席由誰接班？白宮關注這三位歐巴馬時期官員
266,https://news.cnyes.com/news/id/5093050,美歐和其他國家數日內將對俄實施新制裁
267,https://news.cnyes.com/news/id/5093198,台積電跌10元 金融指跌破月線 台股重挫200點
268,https://news.cnyes.com/news/id/5093042,美銀看好Meta驗證訂閱服務 估2024年前訂閱數可達1200萬人


In [57]:
class ctee():
    def __init__(self, url, page:str, start:str = str(int((datetime.datetime.utcnow()-datetime.timedelta(1)).timestamp())), end:str =str(int(datetime.datetime.utcnow().timestamp())-1)):
        self.url = url
        self.page = page
        self.start = start
        self.end = end
        self.prefix = url

    def get_html(self):
        self.response = rq.get(self.url)
        self.soup = bs(self.response.text, 'html.parser')
    
    def get_total_pages(self):
        self.total_pages = 0
        while self.total_pages == 0:
            self.get_html()
            for i in self.soup.find_all('a',{'class':'page-numbers'}):
                if i.text != '下一頁 »':
                    self.total_pages = i.text
            self.total_pages = int(self.total_pages)

    def get_titles_urls_per_page(self):
        data = pd.DataFrame()
        # times = 0
        while len(data) == 0:
            self.get_html()
            links = self.soup.find_all('a',{'style':'color:#777777;text-decoration: none;'})
            urls = []
            titles = []
            for link in links:
                urls.append(link.get('href'))
                titles.append(link.text.strip())
            data = {
                'news_urls':urls,
                'news_titles':titles
            }
            data = pd.DataFrame(data)
            # times+=1
            # print(times)

        return data

    def get_all_titles_urls(self):
        self.get_total_pages()
        data = pd.DataFrame()
        for i in range(1, self.total_pages+1):
            if i == 1:
                temp = self.get_titles_urls_per_page()
            else:
                self.url = self.prefix +'/page/'+str(i)
                temp = self.get_titles_urls_per_page()
            data = pd.concat([data, temp])
        return data



In [58]:
trigger = ctee(ctee_url, 1)

In [59]:
data_ctee = trigger.get_all_titles_urls()
data_ctee

,news_urls,news_titles
0,https://ctee.com.tw/livenews/aj/ctee/A07601002...,Nvidia好嗨 外資讚台積目標價看690元 | 02/23 22:26
1,https://ctee.com.tw/livenews/aj/ctee/A82052002...,鈺齊去年下半年每股配息6元 | 02/23 22:20
2,https://ctee.com.tw/livenews/aj/ctee/A83205002...,數位資產深具潛力 新加坡強化監管建立信任 | 02/23 22:16
3,https://ctee.com.tw/livenews/aj/ctee/A07601002...,大摩重拳降評欣興 目標價砍到100元 | 02/23 22:08
4,https://ctee.com.tw/livenews/aj/ctee/A86127002...,台中五期完熟商圈 建商搶進卡位推案鎖定自住換屋族群 | 02/23 21:41
...,...,...
45,https://ctee.com.tw/livenews/aj/chinatimes/202...,《觀光股》柏文2022年轉盈 每股賺0.95元 | 02/22 07:59
46,https://ctee.com.tw/livenews/aj/chinatimes/202...,《其他電子》朋億*2022年營收、獲利雙創高 EPS11.74元 | 02/22 07:48
47,https://ctee.com.tw/livenews/aj/chinatimes/202...,《興櫃股》證交所董事會通過台灣虎航申請創新板 | 02/22 07:48
48,https://ctee.com.tw/livenews/aj/chinatimes/202...,《電子零件》欣興2022年大賺2股本 股利齊登峰、殖息率達5.82% | 02/22 07:38


In [32]:
# def get_html(url):
#     response = rq.get(url)
#     soup = bs(response.text, 'html.parser')
#     return soup

# def crawl_ctee_titles(url):
#     data = pd.DataFrame()
#     while len(data) == 0:
#         soup = get_html(url)
#         links = soup.find_all('a',{'style':'color:#777777;text-decoration: none;'})
#         urls = []
#         titles = []
#         for link in links:
#             urls.append(link.get('href'))
#             titles.append(link.text.strip())
#         data = {
#             'news_urls':urls,
#             'news_titles':titles
#         }
#         data = pd.DataFrame(data)

#     return data

# def get_ctee_pages(url):
#     final_pages = 0
#     while final_pages == 0:
#         soup = get_html(url)
#         for i in soup.find_all('a',{'class':'page-numbers'}):
#             if i.text != '下一頁 »':
#                 final_pages = i.text
#         final_pages = int(final_pages)
#     return final_pages

# def get_ctee_titles_urls(url):
#     total_pages = get_ctee_pages(url)
#     data = pd.DataFrame()
#     for i in range(1, total_pages+1):
#         if i == 1:
#             temp = crawl_ctee_titles(url)
#         else:
#             new_url = url +'/page/'+str(i)
#             temp = crawl_ctee_titles(new_url)
#         data = pd.concat([data, temp])
#     return data




## 匯入 excel

In [45]:
import openpyxl

In [50]:
# 讀取 excel
workbook = openpyxl.load_workbook('試題.xlsx')
# 選取 試題一 的工作表
sheet = workbook.worksheets[1]
col = ['K']
for index in range(300-13):
    link = data.loc[index, 'news_urls']
    text = data.loc[index, 'news_titles']
    sheet.cell(row=index+13,column=7,value=text)
    sheet.cell(row=index+13,column=7).hyperlink=link

In [51]:
# 儲存檔案
workbook.save('test1.xlsx')